In [10]:
import requests
import re
import operator
from time import sleep
from bs4 import BeautifulSoup
from functools import reduce

domain = "https://www.nikkan.co.jp"
startpath = "/"

In [11]:
basesoup = BeautifulSoup(requests.get(domain + startpath).text, "lxml")

In [12]:
links = []
for a in basesoup.find_all("a", href=re.compile("^/articles/view/[0-9]+")):
    links.append(domain + a.get("href"))
links = list(set(links))

In [13]:
texts = []
for link in links:
    sleep(2)
    res = requests.get(link)
    res.encoding = res.apparent_encoding
    soup = BeautifulSoup(res.text, "lxml")

    title = reduce(operator.add ,[x.text.strip() for x in soup.find_all("div", class_="ttl")], "")
    main = reduce(operator.add ,[x.text.strip() for x in soup.find_all("p", style="text-indent:1em;")], "")
    text = (title + " " + main).replace("\u3000"," ")
    texts.append(text)

In [14]:
import sqlite3
import hashlib
from datetime import datetime

dbname = "text.db"
dbcon = sqlite3.connect(dbname)
dbcur = dbcon.cursor()

for text in texts:
    insert = "INSERT INTO rawtext(id, source, time, rawtext) VALUES(?, ?, ?, ?)"

    id = hashlib.md5(text.encode("utf-8")).hexdigest()
    source = "日刊工業新聞"
    time = datetime.now().isoformat()
    
    args = (id, source, time, text)
    
    try:
        dbcur.execute(insert, args)
    except sqlite3.Error as e:
        print('sqlite3:', e.args[0])
    
dbcon.commit()
dbcon.close()